In [1]:

# !conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge geopy --yes
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows',500)
import json
import requests #
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium # map rendering library
!pip install lxml
import lxml
import time
print('Done!')

Done!


### Get Data from WiKi

In [2]:
# Get data from wiki
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)

In [3]:
def quick_report(df_use,time_s):
    print('Time: {:.2f}\nShape: {}' .format(time.time()-time_s, df_use.shape))

In [4]:
ts=time.time()
df  = pd.DataFrame(data[0]) # new dataframe
df.dropna(axis=0, inplace=True) # Drop all NaN values from dataframe's rows - it's empty Neighborhood and 'Not assigned'Borough
df.reset_index(inplace=True) # reset index
df.drop(columns='index',inplace=True) # delete old index column
print(df.head())
quick_report(df,ts)

  Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
Time: 0.01
Shape: (180, 3)


### Get Data Location from CSV file 

In [12]:
# create DataFrame from csv file with geopoints oh PostalCode 
adr='https://cocl.us/Geospatial_data'
adr_df=pd.read_csv(adr)
adr_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [10]:
ts=time.time()
# Combine both DF by 'Postal Code'
df=pd.merge(df,adr_df, on ='Postal Code', how = 'left')
print(quick_report(df,ts))
df.head()

Time: 0.01
Shape: (180, 7)
None


,Postal Code,Borough,Neighborhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1A,Not assigned,Not assigned,NaN,NaN,NaN,NaN
1,M2A,Not assigned,Not assigned,NaN,NaN,NaN,NaN
2,M3A,North York,Parkwoods,43.753259,-79.329656,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,43.654260,-79.360636


#### Create map of Toronto with Postal Code

In [7]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude,longitude = location.latitude,location.longitude

In [8]:
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers Postal Code to map
for lat, lng, borough, postcode in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Postal Code']):
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.65,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


ValueError: Location values cannot contain NaNs.

#### Ger more information about Toronto from Foursquare service

In [ ]:
CLIENT_ID = 'HBVBLMPIT2KIUOFH3P5LBXXZBZK0XG4WDXQB34WNEHJ3YJRB' # your Foursquare ID (deleted)
CLIENT_SECRET = 'A1FGW51M2O3NWVLBQSMXMW4RJXHQY12J1IPDEYOJVUD0JNDB' # your Foursquare Secret (deleted)
VERSION = '20180605' # Foursquare API version

#### let's explore the top 20 venues that are around each Postal Code within a radius of 1000 meters.

In [27]:
LIMIT = 20
radius = 1000

In [28]:
# Function that will gathering information about category and venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
# Run and get info
ts=time.time()
toronto_venues = getNearbyVenues(names=df['Postal Code'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
quick_report(df,ts)

Time: 24.14
Shape: (103, 5)


In [30]:
toronto_venues.shape

(1089, 7)

#### Let's look at dataframe and how much we have unique category  

In [32]:
print(toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues

(1089, 7)
There are 218 uniques categories.


,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
...,...,...,...,...,...,...,...
1084,M8Z,43.628841,-79.520999,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
1085,M8Z,43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
1086,M8Z,43.628841,-79.520999,Once Upon A Child,43.631075,-79.518290,Kids Store
1087,M8Z,43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


## Analyze Each Postal Code

In [33]:
# one hot encoding and resize the dataframe by mean function
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# # add Postal Code column back to dataframe
# toronto_onehot['Postal Code'] = toronto_venues['Postal Code']
# fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
# toronto_onehot = toronto_onehot[fixed_columns]

# toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
# toronto_grouped.head()

In [34]:
toronto_onehot

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,...,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1085,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1086,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1087,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [622]:
num_top_venues = 10

for PostalCode in toronto_grouped['Postal Code']:
#     print("----"+PostalCode+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == PostalCode].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

In [623]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [624]:
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_venues_sorted = pd.DataFrame(columns=columns)
postal_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    postal_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

### Create K-Means Model and draw segments on map

In [625]:
kclusters = 9
toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([6, 7, 6, 2, 0, 8, 2, 0, 6, 2, 6, 0, 6, 6, 8, 6, 6, 6, 2, 2, 4, 6,
       0, 2, 2, 6, 2, 3, 0, 6, 6, 6, 6, 6, 6, 2, 6, 4, 6, 6, 2, 0, 6, 2,
       2, 3, 2, 3, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 6, 2, 2, 2, 6,
       2, 2, 6, 6, 6, 3, 2, 0, 6, 2, 0, 2, 4, 2, 2, 2, 2, 2, 6, 2, 6, 3,
       1, 6, 2, 6, 1, 4, 6, 6, 6, 7], dtype=int32)

In [626]:
# add clustering labels
postal_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(postal_venues_sorted.set_index('Postal Code'), on='Postal Code')

# we have Postal Codfe without POI :(
toronto_merged['Cluster Labels'].replace(np.nan, kclusters, inplace=True)
toronto_merged.replace(np.nan, 'None', inplace=True)
toronto_merged['Cluster Labels'].value_counts()

In [628]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Some comments about cluster that we have
#### Cluster 0 - Social places
#### Cluster 1 - Sport (Only Baseball Field)
#### Cluster 2 - Relax (Coffee Shop , Fitness)
#### Cluster 3 - Park + Airport
#### Cluster 4 - Park + Shop
#### Cluster 5 - One Garden
#### Cluster 6 - Food court
#### Cluster 7 - Drugstore and Bar
#### Cluster 8 - Playground
#### Cluster 9 - None POI

In [631]:
for i in range (0,kclusters+1):
    print('The Cluster is {}\nThe count of POI for this cluster is {} \nThe popular places in this points are:\n\n{}\n\n'.format(i,toronto_merged[toronto_merged['Cluster Labels'] == i].Borough.count(), 
                              toronto_merged[toronto_merged['Cluster Labels'] == i]['1st Most Common Venue'].value_counts(normalize=True).head(5)))
    print(toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]-9))]].head(5),'\n\n')    

The Cluster is 0
The count of POI for this cluster is 8 
The popular places in this points are:

Park             0.250
Bus Line         0.125
Bank             0.125
Grocery Store    0.125
Bus Stop         0.125
Name: 1st Most Common Venue, dtype: float64


         Borough  Cluster Labels 1st Most Common Venue
0     North York             0.0              Bus Stop
26   Scarborough             0.0                  Bank
31  West Toronto             0.0              Pharmacy
44   Scarborough             0.0              Bus Line
46    North York             0.0         Grocery Store 


The Cluster is 1
The count of POI for this cluster is 2 
The popular places in this points are:

Baseball Field    1.0
Name: 1st Most Common Venue, dtype: float64


        Borough  Cluster Labels 1st Most Common Venue
57   North York             1.0        Baseball Field
101   Etobicoke             1.0        Baseball Field 


The Cluster is 2
The count of POI for this cluster is 39 
The popular places in